## Problem Statement 7
### Implement the Continuous Bag of Words (CBOW) Model for the given (textual document 2) using the below steps: 
    a. Data preparation 
    b. Generate training data 
    c. Train model
    d. Output

In [1]:
from keras.preprocessing import text
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.src.utils import np_utils

from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from keras import backend as K

from sklearn.metrics.pairwise import euclidean_distances

import numpy as np
import pandas as pd

In [9]:
data="""But I must explain to you how all this mistaken idea of denouncing pleasure and praising pain was born and I will give you a complete account of the system, and expound the actual teachings of the great explorer of the truth, the master-builder of human happiness. No one rejects, dislikes, or avoids pleasure itself, because it is pleasure, but because those who do not know how to pursue pleasure rationally encounter consequences that are extremely painful. Nor again is there anyone who loves or pursues or desires to obtain pain of itself, because it is pain, but because occasionally circumstances occur in which toil and pain can procure him some great pleasure. To take a trivial example, which of us ever undertakes laborious physical exercise, except to obtain some advantage from it? But who has any right to find fault with a man who chooses to enjoy a pleasure that has no annoying consequences, or one who avoids a pain that produces no resultant pleasure?"""

dl_data=data.split()

## a. Data preparation

In [11]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(dl_data)

words2id = tokenizer.word_index
words2id['PAD']=0

id2words = {v:k for k,v in words2id.items()}

wids = [[words2id[w] for w in text.text_to_word_sequence(doc)] for doc in dl_data]

vocab_size=len(words2id)
embed_size=100
window_size=2

In [15]:
print("Vocabulary size: ", vocab_size)
print("Vocabulary items: ", list(words2id.items())[:10])

Vocabulary size:  102
Vocabulary items:  [('to', 1), ('of', 2), ('pleasure', 3), ('pain', 4), ('a', 5), ('the', 6), ('who', 7), ('but', 8), ('and', 9), ('or', 10)]


## b. Generating training data

In [21]:
def pairwise(corpus, window_size, vocab_size):
    context_length=window_size*2

    for words in corpus:
        sentence_length=len(words)
        for index, word in enumerate(words):
            context_words=[]
            label_word=[]
            start = index - window_size
            end = index + window_size + 1

            context_words.append([words[i]
                                 for i in range(start, end)
                                 if 0 <= i < sentence_length
                                 and i != index])
            label_word.append(word)

            x = pad_sequences(context_words, maxlen=context_length)
            y = np_utils.to_categorical(label_word, vocab_size)
            yield(x,y)

## c. Training the model

In [22]:
cbow = Sequential()

cbow.add(Embedding(input_dim = vocab_size, output_dim = embed_size, input_length = window_size * 2))
cbow.add(Lambda(lambda x:K.mean(x, axis = 1), output_shape=(embed_size,)))
cbow.add(Dense(vocab_size, activation="softmax"))

cbow.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [23]:
print(cbow.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 4, 100)            10200     
                                                                 
 lambda_1 (Lambda)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 102)               10302     
                                                                 
Total params: 20502 (80.09 KB)
Trainable params: 20502 (80.09 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [32]:
for epoch in range(1,6):
    loss=0

    for x,y in pairwise(corpus=wids, window_size=window_size, vocab_size=vocab_size):
        loss += cbow.train_on_batch(x, y)
    print("Epoch: {} Loss: {}".format(epoch,loss))
    print()

Epoch: 1 Loss: 737.1148734092712

Epoch: 2 Loss: 737.2973875999451

Epoch: 3 Loss: 737.4923667907715

Epoch: 4 Loss: 737.6964325904846

Epoch: 5 Loss: 737.9069187641144



In [34]:
weights = cbow.get_weights()[0]
weights = weights[1:]
print(weights.shape)

pd.DataFrame(weights, index=list(id2words.values())[1:]).head()

(101, 100)


0         1         2         3         4         5   \
of        0.033338  0.016119  0.042000  0.009823  0.006772  0.012067   
pleasure  0.007007  0.030600  0.015222 -0.042876  0.000134 -0.045702   
pain      0.015987  0.047232 -0.030371  0.016748  0.029407  0.027022   
a         0.021878 -0.027201  0.015973  0.021775  0.010877  0.029674   
the      -0.026226 -0.047757  0.038657  0.007546 -0.010249  0.005378   

                6         7         8         9   ...        90        91  \
of       -0.010208  0.027159  0.049581 -0.015044  ...  0.009810  0.026618   
pleasure  0.027381 -0.034875  0.018496  0.015829  ... -0.007084  0.016335   
pain      0.044325  0.016189  0.047599 -0.048728  ... -0.009575  0.007568   
a         0.007320  0.031670  0.021283  0.021772  ... -0.010862  0.042232   
the       0.004800 -0.024317 -0.013206  0.049704  ...  0.002810 -0.024930   

                92        93        94        95        96        97  \
of        0.003227 -0.041501  0.001702  0.004501 -0.046012 -0.009292   
pleasure -0.043365 -0.047175 -0.029347 -0.017310 -0.036235 -0.025692   
pain     -0.036283  0.046010  0.001916 -0.016695 -0.033880 -0.002977   
a         0.039513  0.004146 -0.045278  0.005346  0.034323  0.009638   
the       0.004172  0.001219  0.005802 -0.017852 -0.034751 -0.034388   

                98        99  
of        0.045391 -0.037099  
pleasure  0.001691  0.034270  
pain     -0.037074  0.014130  
a         0.024745 -0.036660  
the       0.013388 -0.034134  

[5 rows x 100 columns]

## d. Output

In [35]:
distance_matrix = euclidean_distances(weights)
print(distance_matrix.shape)

(101, 101)


In [37]:
inwords = input()

similar_words={ search_term: [id2words[idx] for idx in distance_matrix[words2id[search_term]-1].argsort()[0:6]]
              for search_term in {inwords}}
similar_words

 explain


{'explain': ['must', 'how', 'rejects', 'explain', 'desires', 'who']}